In [1]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)


In [3]:
input_query=inputs[1]
input_query

tensor([0.5500, 0.8700, 0.6600])

In [ ]:
input_1=inputs[0]

tensor([0.4300, 0.1500, 0.8900])

In [5]:
torch.dot(input_query, input_1)

tensor(0.9544)

In [8]:
i=0 

res=torch.dot(inputs[i], input_query)
res

tensor(0.9544)